In [2]:
import requests
import json
from math import ceil
from tqdm import tqdm

import time

In [3]:
size = requests.get("https://clinicaltrials.gov/api/v2/stats/size")
total_studies = json.loads(size.content)['totalStudies']

In [3]:
aggregated_studies = []
nextPage = None

request_delay = 1 / 3  #  <= 3 Requests per second

for _ in tqdm(range(ceil(total_studies/1000))):
    try:
        payload = {'format': 'json', 'pageSize': '1000', 'pageToken' : f'{nextPage}'} if nextPage else {'format': 'json', 'pageSize': '1000'}
        data = requests.get("https://clinicaltrials.gov/api/v2/studies", params=payload, timeout=5.0)
        studies = data.json()

        aggregated_studies.extend(studies['studies'])

        # The last page does not have a nextPageToken field
        if 'nextPageToken' not in studies:
            break

        nextPage = studies['nextPageToken']

        time.sleep(request_delay)
    except:
        print('Page Token:', nextPage)
        print('Payload:', payload)
        print('# Aggregated Studies:', len(aggregated_studies))

<class 'dict'>


  1%|▏         | 7/473 [00:06<07:45,  1.00it/s]

<class 'dict'>


  2%|▏         | 8/473 [00:07<07:16,  1.07it/s]

<class 'dict'>


...

  3%|▎         | 13/473 [00:11<06:32,  1.17it/s]

<class 'dict'>


  3%|▎         | 14/473 [00:12<07:03,  1.08it/s]

<class 'dict'>


  3%|▎         | 15/473 [00:13<06:51,  1.11it/s]

In [ ]:
print('Expected Studies:', total_studies)
print('Gathered Studies:', len(aggregated_studies))

In [ ]:
version = requests.get("https://clinicaltrials.gov/api/v2/version").json()
print('Data Time Stamp:', version['dataTimestamp'].split('T')[0])